### Constructing YouTube Links

In [12]:
import pandas as pd
import yt_dlp
from yt_dlp.utils import download_range_func
import concurrent.futures
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(threadName)s - %(levelname)s - %(message)s')

In [3]:
df = pd.read_csv('../data/musiccaps-public.csv')

In [4]:
df.head(1)

,ytid,start_s,end_s,audioset_positive_labels,aspect_list,caption,author_id,is_balanced_subset,is_audioset_eval
0,-0Gj8-vB1q4,30,40,"/m/0140xf,/m/02cjck,/m/04rlf","['low quality', 'sustained strings melody', 's...",The low quality recording features a ballad so...,4,False,True


In [5]:
for row in df.iterrows():
    print(row[1])
    break

ytid                                                              -0Gj8-vB1q4
start_s                                                                    30
end_s                                                                      40
audioset_positive_labels                         /m/0140xf,/m/02cjck,/m/04rlf
aspect_list                 ['low quality', 'sustained strings melody', 's...
caption                     The low quality recording features a ballad so...
author_id                                                                   4
is_balanced_subset                                                      False
is_audioset_eval                                                         True
Name: 0, dtype: object


In [16]:
def download_clip(row):
    video_id = row.ytid
    start_time = row.start_s
    end_time = row.end_s
    index = row.index
    loc = f'../data/audio/audio_{row[0]}'
    
    link = f'https://www.youtube.com/watch?v={video_id}'
    ydl_opts = {
        'outtmpl': loc,
        'quiet': True, # Suppress console output from yt-dlp unless error
        'noplaylist': True, # Ensure only single video is downloaded if ID belongs to a playlist
        'download_ranges': download_range_func(None, [(start_time, end_time)]),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192', # Example quality, e.g., '192' kbps
        }],
    }
    
    print(f"Processing index {index}: Downloading {start_time}-{end_time}s from {link}...")
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([link])
        print(f"Successfully downloaded and processed index {index}")

    except yt_dlp.utils.DownloadError as e:
        print(f"ERROR downloading index {index} ({link}): {e}")
    except Exception as e:
        print(f"An unexpected error occurred for index {index}: {e}")

In [17]:
import time

In [18]:
NUM_THREADS = 10 
results = []

total_items = len(df)
start_run_time = time.time()
logging.info(f"Starting download process for {total_items} items using {NUM_THREADS} threads.")

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    # Use executor.map for a concise way to apply the function to each row
    # It processes rows concurrently and returns results in the original order (if needed)
    # Pass df.itertuples() which yields the named tuples the worker function expects
    future_to_row = {executor.submit(download_clip, row): index for index, row in enumerate(df.itertuples())}

    # Process completed futures as they finish (optional: for progress/results)
    for future in concurrent.futures.as_completed(future_to_row):
        row_index = future_to_row[future]
        try:
            result = future.result() # Get result or raise exception from worker
            results.append(result)
            # Optional: Add progress tracking here if needed
            # logging.info(f"Completed task for index {row_index}: {result}")
        except Exception as exc:
            logging.error(f"Task for index {row_index} generated an exception: {exc}")
            results.append(f"Failed: {row_index}")


end_run_time = time.time()
logging.info(f"Finished processing {total_items} items in {end_run_time - start_run_time:.2f} seconds.")

# Optional: Analyze results
success_count = sum(1 for r in results if r and r.startswith("Success"))
skip_count = sum(1 for r in results if r and r.startswith("Skipped"))
error_count = total_items - success_count - skip_count
logging.info(f"Summary: {success_count} succeeded, {skip_count} skipped, {error_count} failed.")


2025-04-05 17:35:13,427 - MainThread - INFO - Starting download process for 5521 items using 10 threads.


Processing index <built-in method index of Pandas object at 0x7b092b184340>: Downloading 30-40s from https://www.youtube.com/watch?v=-0Gj8-vB1q4...Processing index <built-in method index of Pandas object at 0x7b092b1849c0>: Downloading 30-40s from https://www.youtube.com/watch?v=-0SdAVK79lg...
Processing index <built-in method index of Pandas object at 0x7b092b184a40>: Downloading 30-40s from https://www.youtube.com/watch?v=-0vPFx-wRRI...
Processing index <built-in method index of Pandas object at 0x7b092b184ac0>: Downloading 30-40s from https://www.youtube.com/watch?v=-0xzrMun0Rs...

Processing index <built-in method index of Pandas object at 0x7b092b184bc0>: Downloading 30-40s from https://www.youtube.com/watch?v=-1LrH01Ei1w...
Processing index <built-in method index of Pandas object at 0x7b092b184c40>: Downloading 30-40s from https://www.youtube.com/watch?v=-1OlgJWehn8...
Processing index <built-in method index of Pandas object at 0x7b092b184d40>: Downloading 30-40s from https://www

ERROR: [youtube] -sevczF5etI: Video unavailable


ERROR downloading index <built-in method index of Pandas object at 0x7b092a06dcc0> (https://www.youtube.com/watch?v=-sevczF5etI): ERROR: [youtube] -sevczF5etI: Video unavailable
Processing index <built-in method index of Pandas object at 0x7b092a06dec0>: Downloading 10-20s from https://www.youtube.com/watch?v=-tpq_bzSKes...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7b092a06dc40>
Processing index <built-in method index of Pandas object at 0x7b092a06df40>: Downloading 10-20s from https://www.youtube.com/watch?v=-uaTK8sa5Ms...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7b092a06d840>
Processing index <built-in method index of Pandas object at 0x7b092a077040>: Downloading 30-40s from https://www.youtube.com/watch?v=-v5hgCh3M2w...


ERROR: [youtube] -tpq_bzSKes: Video unavailable. This video is not available


...

ERROR downloading index <built-in method index of Pandas object at 0x7b0929d73ec0> (https://www.youtube.com/watch?v=JNw0A8pRnsQ): ERROR: [youtube] JNw0A8pRnsQ: Video unavailable
Processing index <built-in method index of Pandas object at 0x7b0929d7e0c0>: Downloading 30-40s from https://www.youtube.com/watch?v=JOkuwbhMxbQ...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7b0929d73c40>
Processing index <built-in method index of Pandas object at 0x7b0929d7e140>: Downloading 30-40s from https://www.youtube.com/watch?v=JP637fg_ZC0...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7b0929d739c0>
Processing index <built-in method index of Pandas object at 0x7b0929d7e1c0>: Downloading 30-40s from https://www.youtube.com/watch?v=JPVRBbdykSw...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7b0929d73d40>
Processing index <built-in method index of Pandas object at 0x7b0929d

ERROR: [youtube] Jk2mvFrdZTU: Video unavailable. This video is no longer available due to a copyright claim by Abderrahim Sefiani


ERROR downloading index <built-in method index of Pandas object at 0x7b0929d8c040> (https://www.youtube.com/watch?v=Jk2mvFrdZTU): ERROR: [youtube] Jk2mvFrdZTU: Video unavailable. This video is no longer available due to a copyright claim by Abderrahim Sefiani
Processing index <built-in method index of Pandas object at 0x7b0929d8c1c0>: Downloading 10-20s from https://www.youtube.com/watch?v=JnfMv9ti9Sw...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7b0929d7edc0>
Processing index <built-in method index of Pandas object at 0x7b0929d8c240>: Downloading 30-40s from https://www.youtube.com/watch?v=JoBRbtAnbVM...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7b0929d7ee40>
Processing index <built-in method index of Pandas object at 0x7b0929d8c2c0>: Downloading 60-70s from https://www.youtube.com/watch?v=JorNR3IXDyc...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7

ERROR: [youtube] L5Uu_0xEZg4: Video unavailable


ERROR downloading index <built-in method index of Pandas object at 0x7b0929d35ec0> (https://www.youtube.com/watch?v=L5Uu_0xEZg4): ERROR: [youtube] L5Uu_0xEZg4: Video unavailable
Processing index <built-in method index of Pandas object at 0x7b0929d422c0>: Downloading 380-390s from https://www.youtube.com/watch?v=LAxCq-s84F8...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7b0929d35e40>
Processing index <built-in method index of Pandas object at 0x7b0929d42340>: Downloading 0-10s from https://www.youtube.com/watch?v=LB0u0PrlDHU...
Successfully downloaded and processed index <built-in method index of Pandas object at 0x7b0929d35dc0>
Processing index <built-in method index of Pandas object at 0x7b0929d423c0>: Downloading 30-40s from https://www.youtube.com/watch?v=LByEH80c6Eg...
[download] 100% of  586.06KiB in 00:00:00 at 707.62KiB/sSuccessfully downloaded and processed index <built-in method index of Pandas object at 0x7b0929d42140>
Processing i

In [ ]:
# Define variables
start_time = "00:01:30"
end_time = "00:02:00"
video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
output_location = "/path/to/output/folder/%(title)s.%(ext)s"  # Specify your desired output folder

# Use the variables in the shell command
!yt-dlp --extract-audio --audio-format mp3 --postprocessor-args "-ss {start_time} -to {end_time}" -o "{output_location}" {video_url}


In [ ]:
!yt-dlp --extract-audio --audio-format mp3 --postprocessor-args "-ss 30 -to 40" -o "./audio/test" https://www.youtube.com/watch?v=-0Gj8-vB1q4

[youtube] Extracting URL: https://www.youtube.com/watch?v=-0Gj8-vB1q4
[youtube] -0Gj8-vB1q4: Downloading webpage
[youtube] -0Gj8-vB1q4: Downloading tv client config
[youtube] -0Gj8-vB1q4: Downloading player 73381ccc-main
[youtube] -0Gj8-vB1q4: Downloading tv player API JSON
[youtube] -0Gj8-vB1q4: Downloading ios player API JSON
[youtube] -0Gj8-vB1q4: Downloading m3u8 information
[info] -0Gj8-vB1q4: Downloading 1 format(s): 251
[download] Destination: audio.test

[download]   0.0% of    3.79MiB at  999.83KiB/s ETA 00:03
[download]   0.1% of    3.79MiB at    1.46MiB/s ETA 00:02
[download]   0.2% of    3.79MiB at    3.42MiB/s ETA 00:01
[download]   0.4% of    3.79MiB at    5.85MiB/s ETA 00:00
[download]   0.8% of    3.79MiB at    8.63MiB/s ETA 00:00
[download]   1.6% of    3.79MiB at    8.19MiB/s ETA 00:00
[download]   3.3% of    3.79MiB at    9.53MiB/s ETA 00:00
[download]   6.6% of    3.79MiB at   13.09MiB/s ETA 00:00
[download]  13.2% of    3.79MiB at   15.04MiB/s ETA 00:00
[download] 

ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location
